In [31]:
import os
import numpy as np
import cv2

#### 1. 读取output tensor

In [32]:
# 读取output tensor

"""
call0: 40*40
call1: 20*20
call2: 10*10
"""
def read_output_tensor(path='./20230703/', data_type = 'txt'):
    """_summary_

    Args:
        path (str, optional): _input output tensor path_. Defaults to './20230703/'.
        data_type (str, optional): _output tensor file type_. Defaults to 'txt'.

    Returns:
        _list_: [output tensor1, output tensor2, output tensor3.]
        
    """
    import numpy as np
    output_list = np.sort(os.listdir(path))
    output = []
    for i in output_list:
        if i[-3:] == data_type:
            txt_path = str(path + i)
            with open(txt_path, 'r', encoding='utf-8') as f: # 打开文本
                data = f.read()
                # output.append([float(j) for j in data.strip().split("\n")])
                output.append(np.array(list(map(float, data.strip().split("\n")))))
    print(len(output))
    return output


#### 2. 后处理代码

In [33]:
def letterbox(img, new_shape=(320, 320), color=(0, 0, 0)):  # （自适应图片缩放）这是一种自适应padding的方法，在预处理过程中对原始图像自适应地填充最少的黑边。
    # letterbox函数将输入图像im等比缩放到new_shape大小，不够的地方补充黑边（或者灰边）
    # 流程为：
    # 1. 分别求出长宽的缩放比例，并找到最小值；
    # 2. 按照1中的最小缩放比例对图像个相同性缩放；
    # 3. padding到想要的尺寸；
    
    # Resize and pad image while meeting stride-multiple constraints
    shape = img.shape[:2]  # current shape [height, width]
    if isinstance(new_shape, int):  
        new_shape = (new_shape, new_shape)
        
    # Scale ratio (new / old)
    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    
    # Compute padding
    ratio = r, r  # width, height, ratios
    new_unpad = int(round(shape[1] * r)), int(round(shape[0]*r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  # wh padding
    
    dw /= 2  # divide padding into 2 sides
    dh /= 2 
    
    if shape[::-1] != new_unpad:  # resize 当原始图像与目标shape不一致的时候，进行缩放；
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
        
    # 下面是在处理，dw与dh在上面除以2之前，有可能是个奇数，那么补边的时候，两侧的边补充的像素数应该是相差1的
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    
    # 下面进行两侧的补边操作
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)


In [34]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [42]:
def xywh2xyxy(x):
    # convert [x, y, w, h] to [x1, y1, x2, y2]
    y = np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    return y   

In [43]:
def process(input, mask, anchors):
    anchors = [anchors[i] for i in mask]
    grid_h, grid_w = map(int, input.shape[0:2])

    box_confidence = sigmoid(input[..., 4])
    box_confidence = np.expand_dims(box_confidence, axis=-1)

    box_class_probs = sigmoid(input[..., 5:])

    box_xy = sigmoid(input[..., :2]) * 2 - 0.5

    col = np.tile(np.arange(0, grid_w), grid_w).reshape(-1, grid_w)
    row = np.tile(np.arange(0, grid_h).reshape(-1, 1), grid_h)
    col = col.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    row = row.reshape(grid_h, grid_w, 1, 1).repeat(3, axis=-2)
    grid = np.concatenate((col, row), axis=-1)
    box_xy += grid
    box_xy *= int(IMG_SIZE / grid_h)

    box_wh = pow(sigmoid(input[..., 2:4]) * 2, 2)
    box_wh = box_wh * anchors

    box = np.concatenate((box_xy, box_wh), axis=-1)

    return box, box_confidence, box_class_probs


    

def filter_boxes(boxes, box_confidences, box_class_probs):
    """Filter boxes with box threshold. It's a bit different with origin yolov5 post process!
    # Arguments
        boxes: ndarray, boxes of objects.
        box_confidences: ndarray, confidences of objects.
        box_class_probs: ndarray, class_probs of objects.
    # Returns
        boxes: ndarray, filtered boxes.
        classes: ndarray, classes for boxes.
        scores: ndarray, scores for boxes.
    """
    global BOX_THRESH
    box_classes = np.argmax(box_class_probs, axis=-1)
    box_class_scores = np.max(box_class_probs, axis=-1)
    pos = np.where(box_confidences[..., 0] >= BOX_THRESH)
    
    boxes = boxes[pos]
    classes = box_classes[pos]
    scores = box_class_scores[pos]
    
    return boxes, classes, scores

def nms_boxes(boxes, scores):
    """Suppress non-maximal boxes.

    # Arguments
        boxes: ndarray, boxes of objects.
        scores: ndarray, scores of objects.

    # Returns
        keep: ndarray, index of effective boxes.
    """
    global NMS_THRESH
    
    x = boxes[:, 0]
    y = boxes[:, 1]
    w = boxes[:, 2] - boxes[:, 0]
    h = boxes[:, 3] - boxes[:, 1]
    
    areas = w*h  # 预测框面积；
    order = scores.argsort()[::-1]  # 对scores排序，找到scores最大的; argsort(),返回的是元素值从小到大排序后的索引值的数组，[::-1]则表示从大到小排序；
    
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        
        xx1 = np.maximum(x[i], x[order[1:]])  # 找出最大的x1, np.maximum(a, b) --> a与b逐位比较取其大者
        yy1 = np.maximum(y[i], y[order[1:]])  # 找出最大的y1
        xx2 = np.minimum(x[i] + w[i], x[order[1:]] + w[order[1:]])
        yy2 = np.minimum(y[i] + h[i], y[order[1:]] + h[order[1:]])
        
        w1 = np.maximum(0.0, xx2 - xx1 + 0.00001)
        h1 = np.maximum(0.0, yy2 - yy1 + 0.00001)
        inter = w1 * h1  # 计算交集面积
        
        ovr = inter / (areas[i] + areas[order[1:]] - inter)
        inds = np.where(ovr <= NMS_THRESH)[0]  # 返回交并比小于阈值的索引序列，并取第0个索引。
        order = order[inds + 1]
    
    keep = np.array(keep)
    return keep
        

def yolov5_post_process(input_data):
    boxes, classes, scores = [], [], []
    for input, mask in zip(input_data, masks):
        b, c, s = process(input, mask, anchors)
        b, c, s = filter_boxes(b, c, s)  # filter boxes with box threshold
        boxes.append(b)
        classes.append(c)
        scores.append(s)
    
    boxes = np.concatenate(boxes)
    boxes = xywh2xyxy(boxes)
    classes = np.concatenate(classes)
    scores = np.concatenate(scores)
    
    nboxes, nclasses, nscores = [], [], []
    for c in set(classes):
        inds = np.where(classes == c)
        b = boxes[inds]
        c = classes[inds]
        s = scores[inds]
        
        keep = nms_boxes(b, s)
        
        nboxes.append(b[keep])
        nclasses.append(c[keep])
        nscores.append(s[keep])
        
    if not nclasses and not nscores:
        return None, None, None
    
    boxes = np.concatenate(nboxes)
    classes = np.concatenate(nclasses)
    scores = np.concatenate(nscores)
    
    return boxes, classes, scores    
        

In [51]:
def draw(image, boxes, scores, classes):
    """draw the boxes on the image

    Args:
        image (_type_): original image
        boxes (ndarray): boxes of objects
        scores (ndarray): scores of objects
        classes (ndarray): classes of objects
    """
    global CLASSES
    for box, score, cl in zip(boxes, scores, classes):
        top, left, right, bottom = box
        print('class:{}, score:{}'.format(CLASSES[cl], score))
        print('box coordinate left, top, right, down: [{}, {}, {}, {}]'.format(top, left, right, bottom))
        top = int(top)
        left = int(left)
        right = int(right)
        bottom = int(bottom)
        
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0}{1:.2f}'.format(CLASSES[cl], score), (top, left-6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

In [30]:
if __name__ == "__main__":
    global BOX_THRESH
    global NMS_THRESH
    global IMG_SIZE
    global CLASSES
    global IMG_PATH
    global SAVE_PATH
    
    BOX_THRESH=0.2
    NMS_THRESH=0.6
    IMG_SIZE=320
    CLASSES=("person", "phone")
    IMG_PATH = "./20230703/0_frame_000000000.bmp"
    SAVE_PATH = "./result.png"
    
    
    # 手机检测    
    masks = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
    anchors = [[17, 21], [23, 36], [32, 57], 
               [40, 98], [54, 136], [68, 189], 
               [80, 122], [93, 249], [114, 167]]

    img = cv2.imread(IMG_PATH)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # img, ratio, (dw, dh) = letterbox(img)  # 对img缩放成正方形，并添加黑色边框；
        
    # Inference
    outputs = read_output_tensor(path='./20230703/', data_type = 'txt')   # 读取推理结果；
     
    # post process
    input0_data = outputs[0]
    input1_data = outputs[1]
    input2_data = outputs[2] 
    
    input0_data = input0_data.reshape(40, 3, 7, 40)
    input0_data = np.transpose(input0_data, (1, 2, 0, 3))
    input1_data = input1_data.reshape(10, 3, 7, 10)
    input1_data = np.transpose(input1_data, (1, 2, 0, 3))
    input2_data = input2_data.reshape(20, 3, 7, 20)
    input2_data = np.transpose(input2_data, (1, 2, 0, 3))
    
        
    """
        output tensor dim: anchors * (5+num_cls) * maps_h * mapx_w
        while anchors = 3, 5 means [offset_x, offset_y, offset_w, offset_h, object_confidence]
    """  
    input0_data = input0_data.reshape([3,-1] + list(input0_data.shape[-2:])) 
    input1_data = input1_data.reshape([3,-1] + list(input1_data.shape[-2:]))
    input2_data = input2_data.reshape([3,-1] + list(input2_data.shape[-2:]))

    input_data = list()
    input_data.append(np.transpose(input0_data, (2, 3, 0, 1)))
    input_data.append(np.transpose(input1_data, (2, 3, 0, 1)))
    input_data.append(np.transpose(input2_data, (2, 3, 0, 1)))
        
    print(np.shape(input_data[0]), np.shape(input_data[1]), np.shape(input_data[2]))    
    boxes, classes, scores = yolov5_post_process(input_data)
    print(np.shape(boxes))
    
    img_1 = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    if boxes is not None:
        draw(img_1, boxes, scores, classes)
        
    cv2.imwrite(SAVE_PATH, img_1)

3
(40, 40, 3, 7) (10, 10, 3, 7) (20, 20, 3, 7)
(3353, 4)
class:person, score:nan
box coordinate left, top, right, down: [-12.0, 7.5, 116.0, 64.5]
class:person, score:nan
box coordinate left, top, right, down: [228.0, -8.5, 260.0, 48.5]
class:person, score:nan
box coordinate left, top, right, down: [38.0, 10.0, 130.0, 46.0]
class:person, score:nan
box coordinate left, top, right, down: [251.5, 81.5, 268.5, 102.5]
class:person, score:nan
box coordinate left, top, right, down: [24.5, 34.0, 47.5, 70.0]
class:person, score:nan
box coordinate left, top, right, down: [43.5, 9.5, 60.5, 30.5]
class:person, score:nan
box coordinate left, top, right, down: [43.5, 49.5, 60.5, 70.5]
class:person, score:nan
box coordinate left, top, right, down: [300.0, -8.5, 332.0, 48.5]
class:person, score:nan
box coordinate left, top, right, down: [267.5, 81.5, 284.5, 102.5]
class:person, score:nan
box coordinate left, top, right, down: [12.0, -8.5, 140.0, 48.5]
class:person, score:nan
box coordinate left, top, r